In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("udf").getOrCreate()

22/01/18 16:25:33 WARN Utils: Your hostname, jungheeui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.10 instead (on interface en0)
22/01/18 16:25:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/18 16:25:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/18 16:25:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
transactions = [
    ('찹쌀탕수육+짜장2', '2021-11-07 13:20:00', 22000, 'KRW'),
    ('등심탕수육+크립새우+짜장면', '2021-10-24 11:19:00', 21500, 'KRW'), 
    ('월남 쌈 2인 세트', '2021-07-25 11:12:40', 42000, 'KRW'), 
    ('콩국수+열무비빔국수', '2021-07-10 08:20:00', 21250, 'KRW'), 
    ('장어소금+고추장구이', '2021-07-01 05:36:00', 68700, 'KRW'), 
    ('족발', '2020-08-19 19:04:00', 32000, 'KRW'),  
]

schema = ["name", "datetime", "price", "currency"]

In [4]:
df = spark.createDataFrame(data=transactions, schema=schema)

In [5]:
df.show()

+--------------------------+-------------------+-----+--------+
|                      name|           datetime|price|currency|
+--------------------------+-------------------+-----+--------+
|          찹쌀탕수육+짜장2|2021-11-07 13:20:00|22000|     KRW|
|등심탕수육+크립새우+짜장면|2021-10-24 11:19:00|21500|     KRW|
|          월남 쌈 2인 세트|2021-07-25 11:12:40|42000|     KRW|
|       콩국수+열무비빔국수|2021-07-10 08:20:00|21250|     KRW|
|       장어소금+고추장구이|2021-07-01 05:36:00|68700|     KRW|
|                      족발|2020-08-19 19:04:00|32000|     KRW|
+--------------------------+-------------------+-----+--------+



In [6]:
df.createOrReplaceTempView("transactions")

In [7]:
spark.sql("SELECT * FROM transactions").show()

+--------------------------+-------------------+-----+--------+
|                      name|           datetime|price|currency|
+--------------------------+-------------------+-----+--------+
|          찹쌀탕수육+짜장2|2021-11-07 13:20:00|22000|     KRW|
|등심탕수육+크립새우+짜장면|2021-10-24 11:19:00|21500|     KRW|
|          월남 쌈 2인 세트|2021-07-25 11:12:40|42000|     KRW|
|       콩국수+열무비빔국수|2021-07-10 08:20:00|21250|     KRW|
|       장어소금+고추장구이|2021-07-01 05:36:00|68700|     KRW|
|                      족발|2020-08-19 19:04:00|32000|     KRW|
+--------------------------+-------------------+-----+--------+



In [12]:
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType


In [15]:
def squared(n):
    return n * n

spark.udf.register("squared", squared, LongType())

22/01/18 16:28:49 WARN SimpleFunctionRegistry: The function squared replaced a previously registered function.


<function __main__.squared(n)>

In [16]:
spark.sql("select price, squared(price) from transactions").show()

+-----+--------------+
|price|squared(price)|
+-----+--------------+
|22000|     484000000|
|21500|     462250000|
|42000|    1764000000|
|21250|     451562500|
|68700|    4719690000|
|32000|    1024000000|
+-----+--------------+



In [17]:
def read_number(n):
    units = ["","십","백","천","만"]
    nums = '일이삼사오육칠팔구'
    result = []
    i = 0
    while n>0:
        n,r = divmod(n, 10)
        if r>0:
            result.append(nums[r-1]+units[i])
        i+= 1
    return "".join(result[::-1])


print(read_number(21250))
print(read_number(68700))

이만일천이백오십
육만팔천칠백


In [18]:
spark.udf.register("read_number", read_number)

<function __main__.read_number(n)>

In [19]:
spark.sql("select price, read_number(price) from transactions").show()

+-----+------------------+
|price|read_number(price)|
+-----+------------------+
|22000|          이만이천|
|21500|      이만일천오백|
|42000|          사만이천|
|21250|  이만일천이백오십|
|68700|      육만팔천칠백|
|32000|          삼만이천|
+-----+------------------+



In [20]:
def get_weekday(date):
    import calendar
    return calendar.day_name[date.weekday()]

spark.udf.register("get_weekday", get_weekday)

<function __main__.get_weekday(date)>

In [22]:
query = """
SELECT
    datetime,
    get_weekday(TO_DATE(datetime)) as day_of_week
FROM transactions
"""

spark.sql(query).show()

+-------------------+-----------+
|           datetime|day_of_week|
+-------------------+-----------+
|2021-11-07 13:20:00|     Sunday|
|2021-10-24 11:19:00|     Sunday|
|2021-07-25 11:12:40|     Sunday|
|2021-07-10 08:20:00|   Saturday|
|2021-07-01 05:36:00|   Thursday|
|2020-08-19 19:04:00|  Wednesday|
+-------------------+-----------+

